In [1]:
import pandas as pd
import geopandas as gp
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint

In [2]:
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [3]:
def interchecker(polygon_array):
    """
    
    """
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end

    #the above gives us an array of each possible combination eg
    #combos = [comb[0],comb[1],...]
    #where comb[0] could be (0,1,2,4) or something of the sort
    
    for ele in combos:
        d = {}
        #print(ele) #shows the element used
        #d['region_name'] = region
        #d['index_used'] = ele #for testing
        bucket = []
        for number in ele:
            bucket.append(polygon_array[number])
            #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = np.array(bucket)
        if len(bucket)>1:
            y = bucket[0].intersection(bucket[1])
            for i in range(len(bucket)-2):
                y = y.intersection(bucket[i+2])
            #print(y.area)
            d["combo_area"]= y.area
        else:
            #print(bucket[0].area)
            d['combo_area'] = bucket[0].area
        #for number in ele:
            #d['elements_used'] = polygon_array[number]
            #must move elements used to get every element used- where?
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [4]:
olddata = pd.read_csv("n_obj_df.csv")

In [5]:
olddata.head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield


In [8]:
allregions = olddata.region_name.unique()

In [9]:
allregions

array(['Macclesfield', 'Starburst', 'Manhattan_Classic', 'Wellington',
       'Albany', 'Bilbao', 'Ithaca', 'Portsmouth', 'Pisaq',
       'Manhattan_Frontinella', 'BuenosAires', 'Inca_City_Ridges',
       'Inca_City', 'Giza', 'Potsdam', 'Troy', 'Oswego_Edge', 'Halifax',
       'Caterpillar', 'Rochester', 'Manhattan_Cracks', 'Schenectady',
       'Binghamton', 'Atka', 'Cortland', 'Geneseo'], dtype=object)

In [12]:
for region in allregions:
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['region_name'] = region
        d['Obsid_Used'] = obs
        x = get_outer_polygon_for_obsid(obs)
        area = x.area
        d['Obsid_Area'] = area
        d['polygon_shape'] = x
        bucket.append(d)
    results = pd.DataFrame(bucket)
    
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['obsid'] = obs #we dont want obs, want obs used in combo
        x = get_outer_polygon_for_obsid(obs)
        d['obs_poly'] = x
        bucket.append(d)
    testresult = pd.DataFrame(bucket)
    
    df1 = testresult
    print(testresult)
    print(testresult.obs_poly)
    df2 = interchecker(testresult.obs_poly)
    
    Testregion_inter = interchecker(results.polygon_shape) #all combos of results
    
    bucket2 = []
    for ele in Testregion_inter.index:
        polylist = Testregion_inter.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.obsid[df1['obs_poly'] == poly].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    Testregion_inter['obsids_used'] = np.array(bucket2)

    obsdf = Testregion_inter
    
    bucket = []
    for i in range(len(obsdf.elements_used)): #change to range(len(obsdf.elements_used)) after we check
        #print(len(obsdf.elements_used[i])) #check it's what we want.
        obsdf.at[i, 'number_hirise'] = int(len(obsdf.elements_used[i]))
        #even calling it int(len) still gives float on df, but whatever.
        
        
    bucket2 = []
    for obslist in obsdf.obsids_used:
        bucket = []
        for obs in obslist:
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    obsdf['Ls_used'] = np.array(bucket2)
    obsdf['region_name'] = region

    obsdf.to_csv(f'{region}_duplicate_test_combodf.csv', index = False)

                                             obs_poly            obsid
0   POLYGON ((-67.69881540000002 255.7925763, -68....  ESP_012079_0945
1   POLYGON ((-79.4964947 242.6512937, -71.7773945...  ESP_011407_0945
2   POLYGON ((-72.54717990000002 244.8776493, -72....  ESP_011723_0945
3   POLYGON ((-68.00317840000001 256.1826178, -68....  ESP_011934_0945
4   POLYGON ((-67.989408 256.1117836, -68.32543100...  ESP_012501_0945
5   POLYGON ((-66.91617959999999 257.8571416, -67....  ESP_012290_0945
6   POLYGON ((-66.20375920000001 254.0296356, -66....  ESP_011406_0945
7   POLYGON ((-66.8270493 258.3039062, -66.9954035...  ESP_021494_0945
8   POLYGON ((-66.48888240000001 259.2801681, -66....  ESP_020782_0945
9   POLYGON ((-66.41012389999999 259.1488139, -66....  ESP_020716_0945
10  POLYGON ((-68.1007281 256.0211117, -68.4359608...  ESP_020571_0945
11  POLYGON ((-67.81681570000001 255.7067674, -68....  ESP_020294_0945
12  POLYGON ((-66.3908655 259.2209935, -66.5601174...  ESP_020927_0945
13  PO

TypeError: float() argument must be a string or a number, not 'Point'

In [16]:
olddata[olddata['obsid'] == 'ESP_012884_0935'] #problem is in manhattan_classic

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
72,-86.3912,254.194,29,both,0.098,681,ESP_012884_0935,3.588417e+06,Manhattan_Classic
80,-86.3912,254.194,29,fans,0.050,408,ESP_012884_0935,1.837569e+06,Manhattan_Classic
88,-86.3912,254.194,29,blotches,0.056,337,ESP_012884_0935,2.059682e+06,Manhattan_Classic


In [31]:
olddata[olddata['region_name'] == 'Macclesfield'].head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield


In [25]:
test = olddata[olddata['region_name'] == 'Manhattan_Classic']

In [45]:
get_outer_polygon_for_obsid('ESP_012884_0935')

Michael's code is broken for manhattan classic, lets try to see why

In [47]:
tilestest = pd.read_csv("tile_coordinates.csv")

In [51]:
tilestest[tilestest['obsid'] == 'ESP_012884_0935' ].head()

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
4777,1,1,420.0,324.0,-86.455091,-86.496595,NaN,NaN,NaN,NaN,APF0000dsd,ESP_012884_0935
4778,1,2,420.0,872.0,-86.452087,-86.493626,NaN,NaN,NaN,NaN,APF0000dqa,ESP_012884_0935
4779,1,3,420.0,1420.0,-86.449083,-86.490657,NaN,NaN,NaN,NaN,APF0000dru,ESP_012884_0935
4780,1,4,420.0,1968.0,-86.446077,-86.487686,NaN,NaN,NaN,NaN,APF0000dpx,ESP_012884_0935
4781,1,5,420.0,2516.0,-86.443066,-86.484709,NaN,NaN,NaN,NaN,APF0000dsb,ESP_012884_0935


Can see for these tiles there's no x, y, z coords.  Am I on an old version of this or is this another broken df?  Figure out how to update to michael's current version, and if this is it let him know it's still broken.  I have planetfour-catalog v 1.1.2, ask him how current that is.